### Problem 1

#### Set up for dataset and model

Package installation, loading, and dataloaders. There's also a resnet18 model defined.

In [1]:
# !pip install tensorboardX

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt
from tqdm import tqdm

from torchvision import datasets, transforms
# from tensorboardX import SummaryWriter

use_cuda = True
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.CIFAR10('cifar10_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.CIFAR10('cifar10_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)



Files already downloaded and verified
Files already downloaded and verified


In [2]:

def tp_relu(x, delta=1.):
    ind1 = (x < -1. * delta).float()
    ind2 = (x > delta).float()
    return .5 * (x + delta) * (1 - ind1) * (1 - ind2) + x * ind2

def tp_smoothed_relu(x, delta=1.):
    ind1 = (x < -1. * delta).float()
    ind2 = (x > delta).float()
    return (x + delta) ** 2 / (4 * delta) * (1 - ind1) * (1 - ind2) + x * ind2

class Normalize(nn.Module):
    def __init__(self, mu, std):
        super(Normalize, self).__init__()
        self.mu, self.std = mu, std

    def forward(self, x):
        return (x - self.mu) / self.std

class IdentityLayer(nn.Module):
    def forward(self, inputs):
        return inputs
    
class PreActBlock(nn.Module):
    '''Pre-activation version of the BasicBlock.'''
    expansion = 1

    def __init__(self, in_planes, planes, bn, learnable_bn, stride=1, activation='relu'):
        super(PreActBlock, self).__init__()
        self.collect_preact = True
        self.activation = activation
        self.avg_preacts = []
        self.bn1 = nn.BatchNorm2d(in_planes, affine=learnable_bn) if bn else IdentityLayer()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=not learnable_bn)
        self.bn2 = nn.BatchNorm2d(planes, affine=learnable_bn) if bn else IdentityLayer()
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=not learnable_bn)

        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=not learnable_bn)
            )

    def act_function(self, preact):
        if self.activation == 'relu':
            act = F.relu(preact)
        elif self.activation[:6] == '3prelu':
            act = tp_relu(preact, delta=float(self.activation.split('relu')[1]))
        elif self.activation[:8] == '3psmooth':
            act = tp_smoothed_relu(preact, delta=float(self.activation.split('smooth')[1]))
        else:
            assert self.activation[:8] == 'softplus'
            beta = int(self.activation.split('softplus')[1])
            act = F.softplus(preact, beta=beta)
        return act

    def forward(self, x):
        out = self.act_function(self.bn1(x))
        shortcut = self.shortcut(out) if hasattr(self, 'shortcut') else x  # Important: using out instead of x
        out = self.conv1(out)
        out = self.conv2(self.act_function(self.bn2(out)))
        out += shortcut
        return out

class PreActResNet(nn.Module):
    def __init__(self, block, num_blocks, n_cls, cuda=True, half_prec=False,
        activation='relu', fts_before_bn=False, normal='none'):
        super(PreActResNet, self).__init__()
        self.bn = True
        self.learnable_bn = True  # doesn't matter if self.bn=False
        self.in_planes = 64
        self.avg_preact = None
        self.activation = activation
        self.fts_before_bn = fts_before_bn
        if normal == 'cifar10':
            self.mu = torch.tensor((0.4914, 0.4822, 0.4465)).view(1, 3, 1, 1)
            self.std = torch.tensor((0.2471, 0.2435, 0.2616)).view(1, 3, 1, 1)
        else:
            self.mu = torch.tensor((0.0, 0.0, 0.0)).view(1, 3, 1, 1)
            self.std = torch.tensor((1.0, 1.0, 1.0)).view(1, 3, 1, 1)
            print('no input normalization')
        if cuda:
            self.mu = self.mu.cuda()
            self.std = self.std.cuda()
        if half_prec:
            self.mu = self.mu.half()
            self.std = self.std.half()

        self.normalize = Normalize(self.mu, self.std)
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=not self.learnable_bn)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.bn = nn.BatchNorm2d(512 * block.expansion)
        self.linear = nn.Linear(512*block.expansion, n_cls)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, self.bn, self.learnable_bn, stride, self.activation))
            # layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, return_features=False):
        for layer in [*self.layer1, *self.layer2, *self.layer3, *self.layer4]:
            layer.avg_preacts = []

        out = self.normalize(x)
        out = self.conv1(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        if return_features and self.fts_before_bn:
            return out.view(out.size(0), -1)
        out = F.relu(self.bn(out))
        if return_features:
            return out.view(out.size(0), -1)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)

        return out


def PreActResNet18(n_cls, cuda=True, half_prec=False, activation='relu', fts_before_bn=False,
    normal='none'):
    #print('initializing PA RN-18 with act {}, normal {}'.format())
    return PreActResNet(PreActBlock, [2, 2, 2, 2], n_cls=n_cls, cuda=cuda, half_prec=half_prec,
        activation=activation, fts_before_bn=fts_before_bn, normal=normal)


#### Implement the Attacks

Functions are given a simple useful signature that you can start with. Feel free to extend the signature as you see fit.

You may find it useful to create a 'batched' version of PGD that you can use to create the adversarial attack.

In [3]:
def pgd_linf_untargeted(model, x, labels, k, eps, eps_step):
    model.eval()
    ce_loss = torch.nn.CrossEntropyLoss()
    adv_x = x.clone().detach().to(device)
    adv_x.requires_grad_(True) 
    for _ in range(k):
        adv_x.requires_grad_(True)
        model.zero_grad()
        output = model(adv_x)
        # TODO: Calculate the loss
        loss = ce_loss(output, labels)
        loss.backward()
        # TODO: compute the adv_x
        # find delta, clamp with eps
        with torch.no_grad():
            adv_x = adv_x + eps_step * adv_x.grad.sign()
            delta = adv_x - x
            delta = torch.clamp(delta, -eps, eps)
            adv_x = torch.clamp(x + delta, min=0, max=1).detach()
    return adv_x

In [4]:
def pgd_l2_untargeted(model, x, labels, k, eps, eps_step):
    model.eval()
    ce_loss = torch.nn.CrossEntropyLoss()
    adv_x = x.clone().detach()
    adv_x.requires_grad_(True) 
    for _ in range(k):
        adv_x.requires_grad_(True)
        model.zero_grad()
        output = model(adv_x)
        batch_size = x.size()[0]
        # TODO: Calculate the loss
        loss = ce_loss(output, labels)
        loss.backward()
        grad = adv_x.grad.data
        # TODO: compute the adv_x
        # find delta, clamp with eps, project delta to the l2 ball
        # HINT: https://github.com/Harry24k/adversarial-attacks-pytorch/blob/master/torchattacks/attacks/pgdl2.py
        with torch.no_grad():
            grad_norms = (
                torch.norm(grad.view(batch_size, -1), p=2, dim=1) + 1e-10
            )
            adv_x = adv_x + eps_step * grad / grad_norms.view(-1, 1, 1, 1)
            delta = adv_x - x
            delta_norms = torch.norm(delta.view(batch_size, -1), p=2, dim=1)
            factor = eps / delta_norms
            factor = torch.min(factor, torch.ones_like(delta_norms))
            delta = delta * factor.view(-1, 1, 1, 1)
            adv_x = torch.clamp(x + delta, min=0, max=1).detach()

    return adv_x

### PDG-based Adversarial Training

In [10]:
def adversarial_train(
    model, train_loader, optimizer, epoch, k, eps, attack_type="original"
):
    model.train()
    ce_loss = torch.nn.CrossEntropyLoss()
    scheduler = optim.lr_scheduler.MultiStepLR(
        optimizer, milestones=[epoch // 2, (3 * epoch) // 4], gamma=0.1
    )
    if attack_type == "original":
        attack_func = lambda model, x, y, k, eps, eps_step: x
    elif attack_type == "pgd_l2":
        attack_func = pgd_l2_untargeted
    elif attack_type == "pgd_linf":
        attack_func = pgd_linf_untargeted
        
    for e in range(epoch):
        progress_bar = tqdm(train_loader)  # Move this inside the epoch loop
        for batch_idx, (data, target) in enumerate(progress_bar):
            data, target = data.to(device), target.to(device)
            adv_data = attack_func(model, data, target, k, eps, eps / 4)
            model.train()
            optimizer.zero_grad()
            output = model(adv_data)
            loss = ce_loss(output, target)
            loss.backward()
            optimizer.step()
            progress_bar.set_description(
                f"Epoch {e+1} | Batch {batch_idx+1}/{len(train_loader)} | Loss: {loss.item():.4f}"
            )
        scheduler.step()

#### Evaluate Single and Multi-Norm Robust Accuracy

In this section, we evaluate the model on the Linf and L2 attacks as well as union accuracy.

In [6]:
def test_model_on_single_attack(model, attack="pgd_linf", k=10, eps=0.1):
    model.eval()
    tot_test, tot_acc, orig_acc = 0.0, 0.0, 0.0
    for batch_idx, (x_batch, y_batch) in tqdm(
        enumerate(test_loader), total=len(test_loader), desc="Evaluating"
    ):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        if attack == "pgd_linf":
            x_adv = pgd_linf_untargeted(
                model, x_batch, y_batch, k=k, eps=eps, eps_step=eps / 4
            )
        elif attack == "pgd_l2":
            x_adv = pgd_l2_untargeted(
                model, x_batch, y_batch, k=k, eps=eps, eps_step=eps / 4
            )
        else:
            x_adv = x_batch

        # get the testing accuracy and update tot_test and tot_acc
        with torch.no_grad():
            out = model(x_adv)
            tot_acc += (out.argmax(dim=1) == y_batch).sum().item()
            tot_test += x_batch.size(0)
            orig_acc += (model(x_batch).argmax(dim=1) == y_batch).sum().item()

    print(
        "Standard accuracy %.5lf" % (orig_acc / tot_test),
        "Robust accuracy %.5lf" % (tot_acc / tot_test),
        f"on {attack} attack with eps = {eps}",
    )
    return orig_acc / tot_test, tot_acc / tot_test

## Model Training and Evaluation

In [11]:
import json

attacks = ["pgd_linf"]
epsilons = [2 / 255, 4 / 255, 8 / 255]
epoch = 2

results = {}

print("Training model with original data")
model = PreActResNet18(10, cuda=True, activation="softplus1").to(device)

adversarial_train(model, train_loader, optim.SGD(model.parameters(), lr=0.1,
    momentum=0.9, weight_decay=5e-4), epoch=epoch, k=10, eps=8/255, attack_type='original')
torch.save(model.state_dict(), './models/model_original.pth')

for attack in attacks:
    for eps_val in epsilons:
        print(f"Training model with {attack} attack and eps {eps_val}")
        model = PreActResNet18(10, cuda=True, activation="softplus1").to(device)

        adversarial_train(model, train_loader, optim.SGD(model.parameters(), lr=0.1,
            momentum=0.9, weight_decay=5e-4), epoch=epoch, k=10, eps=eps_val, attack_type=attack)
        orig_acc, robust_acc = test_model_on_single_attack(model, attack=attack, k=10, eps=eps_val)
        torch.save(model.state_dict(), f'./models/model_{attack}_eps{int(eps_val * 255)}.pth')
        results[f'{attack}_eps{int(eps_val * 255)}'] = {'orig_acc': orig_acc, 'robust_acc': robust_acc}

# Save results to file
with open('training_results.json', 'w') as f:
    json.dump(results, f, indent=2)

print("Results saved to training_results.json")

Training model with original data
no input normalization


Epoch 1 | Batch 782/782 | Loss: 1.5050: 100%|██████████| 782/782 [00:50<00:00, 15.54it/s]
Epoch 2 | Batch 782/782 | Loss: 1.0727: 100%|██████████| 782/782 [00:51<00:00, 15.25it/s]


Training model with pgd_linf attack and eps 0.00784313725490196
no input normalization


Epoch 1 | Batch 782/782 | Loss: 2.0554: 100%|██████████| 782/782 [07:24<00:00,  1.76it/s]
Epoch 2 | Batch 782/782 | Loss: 1.1615: 100%|██████████| 782/782 [07:25<00:00,  1.76it/s]
Evaluating: 100%|██████████| 157/157 [01:25<00:00,  1.83it/s]


Standard accuracy 0.48600 Robust accuracy 0.40380 on pgd_linf attack with eps = 0.00784313725490196
Training model with pgd_linf attack and eps 0.01568627450980392
no input normalization


Epoch 1 | Batch 782/782 | Loss: 1.6302: 100%|██████████| 782/782 [07:25<00:00,  1.76it/s]
Epoch 2 | Batch 782/782 | Loss: 1.5671: 100%|██████████| 782/782 [07:26<00:00,  1.75it/s]
Evaluating: 100%|██████████| 157/157 [01:25<00:00,  1.84it/s]


Standard accuracy 0.47180 Robust accuracy 0.36090 on pgd_linf attack with eps = 0.01568627450980392
Training model with pgd_linf attack and eps 0.03137254901960784
no input normalization


Epoch 1 | Batch 782/782 | Loss: 1.8665: 100%|██████████| 782/782 [07:24<00:00,  1.76it/s]
Epoch 2 | Batch 782/782 | Loss: 2.1642: 100%|██████████| 782/782 [07:24<00:00,  1.76it/s]
Evaluating: 100%|██████████| 157/157 [01:25<00:00,  1.84it/s]


Standard accuracy 0.41770 Robust accuracy 0.28700 on pgd_linf attack with eps = 0.03137254901960784
Results saved to training_results.json


In [12]:
model = PreActResNet18(10, cuda=True, activation="softplus1").to(device)
model.load_state_dict(torch.load('./models/model_original.pth'))
for eps in epsilons:
    test_model_on_single_attack(model, attack="pgd_linf", k=1, eps=eps)
model.load_state_dict(torch.load('./models/model_pgd_linf_eps8.pth'))
for eps in epsilons:
    test_model_on_single_attack(model, attack="pgd_linf", k=1, eps=eps)
model.load_state_dict(torch.load('./models/model_pgd_linf_eps4.pth'))
for eps in epsilons:
    test_model_on_single_attack(model, attack="pgd_linf", k=1, eps=eps) 
model.load_state_dict(torch.load('./models/model_pgd_linf_eps2.pth'))
for eps in epsilons:
    test_model_on_single_attack(model, attack="pgd_linf", k=1, eps=eps)


no input normalization


Evaluating: 100%|██████████| 157/157 [00:14<00:00, 11.11it/s]


Standard accuracy 0.51760 Robust accuracy 0.46940 on pgd_linf attack with eps = 0.00784313725490196


Evaluating: 100%|██████████| 157/157 [00:14<00:00, 11.13it/s]


Standard accuracy 0.51760 Robust accuracy 0.42420 on pgd_linf attack with eps = 0.01568627450980392


...

Standard accuracy 0.48600 Robust accuracy 0.46430 on pgd_linf attack with eps = 0.00784313725490196


Evaluating: 100%|██████████| 157/157 [00:14<00:00, 11.05it/s]


Standard accuracy 0.48600 Robust accuracy 0.44370 on pgd_linf attack with eps = 0.01568627450980392


Evaluating: 100%|██████████| 157/157 [00:14<00:00, 11.05it/s]

Standard accuracy 0.48600 Robust accuracy 0.40360 on pgd_linf attack with eps = 0.03137254901960784


### Problem 2

In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import trange
import time

from bound_propagation import BoundModelFactory, HyperRectangle


class SimpleNet(nn.Sequential):
    def __init__(self):
        super(SimpleNet, self).__init__(
            nn.Linear(28 * 28, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, 10),
        )

    def forward(self, x):
        x = x = x.view(x.size(0), -1)
        return super().forward(x)


def get_logit(bounds, y):
    batch_size = y.size(0)
    logit = bounds.upper.clone()
    logit[torch.arange(batch_size), y] = bounds.lower[torch.arange(batch_size), y]
    return logit

In [ ]:
def train_ibp(model, lr=1e-3, num_epochs=100, k_final=0.5, epsilon_target=0.1):

    model.to(device)
    model.train()
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
    train_data = datasets.MNIST(
        "./data", train=True, download=True, transform=transform
    )
    train_loader = DataLoader(train_data, batch_size=100, shuffle=True, num_workers=4)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    total_steps = num_epochs * len(train_loader)
    warm_up = int(2 / 60 * total_steps)
    ramp_up = int(10 / 60 * total_steps)
    lr_decay_steps = [int(15 / 60 * total_steps), int(25 / 60 * total_steps)]

    start_time = time.time()
    current_step = 0

    for epoch in range(num_epochs):
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")

        for batch_idx, (data, target) in enumerate(progress_bar):
            data, target = data.to(device), target.to(device)

            if current_step in lr_decay_steps:
                for param_group in optimizer.param_groups:
                    param_group["lr"] *= 0.1

            if current_step < warm_up:
                lr_scale = current_step / warm_up
                for param_group in optimizer.param_groups:
                    param_group["lr"] = lr * lr_scale

            k = max(
                1.0
                - (1.0 - k_final)
                * max(0, current_step - warm_up)
                / (total_steps - warm_up),
                k_final,
            )

            if current_step < ramp_up:
                eps_train = epsilon_target * (current_step / ramp_up)
            else:
                eps_train = epsilon_target

            optimizer.zero_grad(set_to_none=True)

            y_hat = model(data)
            ce_loss = criterion(y_hat, target)

            bounds = model.ibp(HyperRectangle.from_eps(data.view(data.size(0), -1), eps_train))
            adv_logit = get_logit(bounds, target)
            robust_loss = criterion(adv_logit, target)

            loss = k * ce_loss + (1 - k) * robust_loss
            loss.backward()
            optimizer.step()

            current_step += 1

            progress_bar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'CE_Loss': f'{ce_loss.item():.4f}',
                'Rob_Loss': f'{robust_loss.item():.4f}',
                'k': f'{k:.3f}',
                'eps': f'{eps_train:.3f}'
            })

    training_time = time.time() - start_time
    print(f"IBP Training completed in {training_time:.2f} seconds")


In [39]:
model = SimpleNet()
bound_model = BoundModelFactory().build(model)
bound_model.to(device)
train_ibp(bound_model, lr=1e-3, num_epochs=20, k_final=0.5, epsilon_target=0.1)

Epoch 20: 100%|██████████| 600/600 [00:18<00:00, 32.28it/s, Loss=0.5480, CE_Loss=0.2017, Rob_Loss=0.8943, k=0.500, eps=0.100]

IBP Training completed in 370.46 seconds


370.4572105407715

In [42]:
def train_standard(model, lr=1e-3, num_epochs=100):

    model.to(device)
    model.train()
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
    )
    train_data = datasets.MNIST(
        "./data", train=True, download=True, transform=transform
    )
    train_loader = DataLoader(train_data, batch_size=100, shuffle=True, num_workers=4)

    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    start_time = time.time()
    for epoch in range(num_epochs):
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}")

        for batch_idx, (data, target) in enumerate(progress_bar):
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad(set_to_none=True)

            y_hat = model(data)
            ce_loss = criterion(y_hat, target)

            loss = ce_loss
            loss.backward()
            optimizer.step()

            progress_bar.set_postfix(
                {
                    "Loss": f"{loss.item():.4f}",
                }
            )

    training_time = time.time() - start_time
    print(f"Standard Training completed in {training_time:.2f} seconds")
    return training_time

In [43]:
standard_model = SimpleNet().to(device)
train_standard(standard_model, lr=1e-3, num_epochs=20)

Epoch 20: 100%|██████████| 600/600 [00:10<00:00, 57.30it/s, Loss=0.0227] 

Standard Training completed in 218.18 seconds


218.17817997932434

In [56]:
def interval_analysis(net, epsilons):

    mnist_test_loader = DataLoader(
        datasets.MNIST(
            "./data",
            train=False,
            download=True,
            transform=transforms.Compose(
                [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
            ),
        ),
        batch_size=100,
        shuffle=False
    )
    
    with torch.no_grad():
        for e in epsilons:
            correct = 0
            total = 0
            progress_bar = tqdm(mnist_test_loader, desc=f"Epsilon {e:.3f}")
            for imgs, labels in progress_bar:
                box = HyperRectangle.from_eps(imgs.view(imgs.size(0), -1), e)
                out = net.crown_ibp(box, alpha=True).concretize()
                lo, hi = out.lower, out.upper
                labs = labels.cpu().numpy()
                bad_mask = torch.ones_like(hi, dtype=torch.bool, device=hi.device)
                for idx, lab in enumerate(labs):
                    bad_mask[idx, lab] = False
                lo_true = lo[range(len(labels)), labs]
                hi_bad = hi.masked_select(bad_mask).view(len(labels), -1)
                correct += (lo_true > hi_bad.max(dim=1).values).sum().item()
                total += len(labels)
            print(f"Epsilon: {e}, Robust Accuracy: {100 * correct / total:.2f}%")

In [45]:
def test_bound_model_on_single_attack(model, attack="pgd_linf", k=10, eps=0.1):

    bound_model.eval()
    test_loader = DataLoader(
        datasets.MNIST(
            "./data",
            train=False,
            download=True,
            transform=transforms.Compose(
                [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
            ),
        )
    )
    tot_test, tot_acc, orig_acc = 0.0, 0.0, 0.0
    for batch_idx, (x_batch, y_batch) in tqdm(
        enumerate(test_loader), total=len(test_loader), desc="Evaluating"
    ):
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        if attack == "pgd_linf":
            x_adv = pgd_linf_untargeted(
                model, x_batch, y_batch, k=k, eps=eps, eps_step=eps / 4
            )
        elif attack == "pgd_l2":
            x_adv = pgd_l2_untargeted(
                model, x_batch, y_batch, k=k, eps=eps, eps_step=eps / 4
            )
        else:
            x_adv = x_batch

        # get the testing accuracy and update tot_test and tot_acc
        with torch.no_grad():
            out = model(x_adv)
            tot_acc += (out.argmax(dim=1) == y_batch).sum().item()
            tot_test += x_batch.size(0)
            orig_acc += (model(x_batch).argmax(dim=1) == y_batch).sum().item()

    print(
        "Standard accuracy %.5lf" % (orig_acc / tot_test),
        "Robust accuracy %.5lf" % (tot_acc / tot_test),
        f"on {attack} attack with eps = {eps}",
    )

In [46]:
test_bound_model_on_single_attack(bound_model, attack="pgd_linf", k=10, eps=0.1)

Evaluating: 100%|██████████| 10000/10000 [04:14<00:00, 39.33it/s]


Standard accuracy 0.95970 Robust accuracy 0.83280 on pgd_linf attack with eps = 0.1


In [ ]:
interval_analysis(bound_model.to('cpu'), epsilons=np.linspace(0.01, 0.1, 10))

Epsilon 0.010: 100%|██████████| 100/100 [00:57<00:00,  1.73it/s]


Epsilon: 0.01, Robust Accuracy: 95.57%


Epsilon 0.020: 100%|██████████| 100/100 [00:57<00:00,  1.73it/s]


Epsilon: 0.020000000000000004, Robust Accuracy: 95.02%


Epsilon 0.030: 100%|██████████| 100/100 [00:57<00:00,  1.73it/s]


...

Epsilon 0.090: 100%|██████████| 100/100 [01:01<00:00,  1.63it/s]


Epsilon: 0.09000000000000001, Robust Accuracy: 87.54%


Epsilon 0.100: 100%|██████████| 100/100 [01:01<00:00,  1.63it/s]

Epsilon: 0.1, Robust Accuracy: 85.99%


: 